### 1. Import libraries and data

In [2]:
import pandas as pd
from shapely.geometry import Point, shape
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import folium
from folium import Choropleth, LayerControl
import json

In [4]:
# This command prompts matplotlib visuals to appear in the notebook 
%matplotlib inline

In [6]:
# Import ".json" file for the U.S. 
with open('C:/Users/lourd/A6_Open Project/Data Sets/us-states.json') as f:
    us_states = json.load(f)

In [8]:
# Look at the JSON file contents
f = open(r'C:/Users/lourd/A6_Open Project/Data Sets/us-states.json',)

In [10]:
# returns JSON object asa dictionary
data = json.load(f)

In [12]:
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'id': 'AL', 'properties': {'name': 'Alabama'}, 'geometry': {'type': 'Polygon', 'coordinates': [[[-87.359296, 35.00118], [-85.606675, 34.984749], [-85.431413, 34.124869], [-85.184951, 32.859696], [-85.069935, 32.580372], [-84.960397, 32.421541], [-85.004212, 32.322956], [-84.889196, 32.262709], [-85.058981, 32.13674], [-85.053504, 32.01077], [-85.141136, 31.840985], [-85.042551, 31.539753], [-85.113751, 31.27686], [-85.004212, 31.003013], [-85.497137, 30.997536], [-87.600282, 30.997536], [-87.633143, 30.86609], [-87.408589, 30.674397], [-87.446927, 30.510088], [-87.37025, 30.427934], [-87.518128, 30.280057], [-87.655051, 30.247195], [-87.90699, 30.411504], [-87.934375, 30.657966], [-88.011052, 30.685351], [-88.10416, 30.499135], [-88.137022, 30.318396], [-88.394438, 30.367688], [-88.471115, 31.895754], [-88.241084, 33.796253], [-88.098683, 34.891641], [-88.202745, 34.995703], [-87.359296, 35.00118]]]}}
{'type': 'Feature', 'id': 'AK', 'properties': {'name': 'Alaska'},

In [14]:
#Import my data
file_path = 'C:/Users/lourd/A6_Open Project/Data Sets/frpp_data_subset_cleaned.xlsx'

In [16]:
frpp_data = pd.read_excel(file_path)

In [18]:
frpp_data.head()

,Latitude,Longitude,Real Property Type,Real Property Use,Utilization,Utilization Code,Asset Status,Asset Status Code,Building Age,Number of Federal Employees,Owned and Otherwise Managed Annual Operations Cost,Owned and Otherwise Managed Annual Maintenance Cost,Square Feet (Buildings),Year of Construction
0,34.24330,-79.79580,Building,Laboratories,Utilized,6,Current Mission Need,A,60,23,1726.41,5179.24,16750,0
1,32.82920,-83.73550,Building,Laboratories,Utilized,6,Current Mission Need,A,60,18,1726.41,5179.24,22524,0
2,41.84585,-87.67311,Building,Office,Utilized,6,Current Mission Need,A,60,13,1726.41,5179.24,2175,0
3,28.02150,-81.73050,Building,Laboratories,Utilized,6,Current Mission Need,A,60,91,1726.41,5179.24,7335,0
4,35.27560,-81.14500,Building,Laboratories,Utilized,6,Current Mission Need,A,60,17,1726.41,5179.24,10400,0


In [20]:
frpp_data.shape

(9952, 14)

In [22]:
frpp_data.columns

Index(['Latitude', 'Longitude', 'Real Property Type', 'Real Property Use',
       'Utilization', 'Utilization Code', 'Asset Status', 'Asset Status Code',
       'Building Age', 'Number of Federal Employees',
       'Owned and Otherwise Managed Annual Operations Cost',
       'Owned and Otherwise Managed Annual Maintenance Cost',
       'Square Feet (Buildings)', 'Year of Construction'],
      dtype='object')

In [24]:
# Create an empty dictionary for mapping
mapping_dict = {}

In [26]:
# My data in Excel file has 'Latitude' and 'Longitude' columns
for index, row in frpp_data.iterrows():   
    point = Point(row['Longitude'], row['Latitude'])
    
    # Loop through each state in the JSON to find the state containing the point
    for feature in us_states['features']:
        polygon = shape(feature['geometry'])
        try:
            if polygon.buffer(0).contains(point):   
                state_abbr = feature['id']
                mapping_dict[index] = state_abbr  # Map the row index to the state abbreviation
                break
        except Exception as e:
            print(f"Error with polygon {feature['id']}: {e}")

In [28]:
# Print or use the mapping dictionary
print(mapping_dict)

{0: 'SC', 1: 'GA', 2: 'IL', 3: 'FL', 4: 'NC', 5: 'NC', 6: 'TX', 7: 'MD', 8: 'MD', 9: 'MD', 10: 'MI', 11: 'ND', 12: 'MI', 13: 'MI', 14: 'MI', 15: 'MI', 16: 'MI', 17: 'MI', 18: 'MS', 19: 'MS', 20: 'AL', 21: 'KY', 22: 'TX', 23: 'ND', 24: 'OK', 25: 'TX', 26: 'TX', 27: 'CO', 28: 'TX', 29: 'TN', 30: 'MS', 31: 'MS', 32: 'MS', 33: 'MS', 34: 'IN', 35: 'MT', 36: 'MT', 37: 'MT', 38: 'UT', 39: 'MT', 40: 'NE', 41: 'NE', 42: 'ND', 43: 'MD', 44: 'MD', 45: 'MD', 46: 'MD', 47: 'MD', 48: 'MD', 49: 'MD', 50: 'MD', 51: 'GA', 52: 'MD', 53: 'NC', 54: 'MD', 55: 'MD', 56: 'ME', 57: 'ME', 58: 'DE', 59: 'TX', 60: 'TX', 61: 'TX', 62: 'OK', 63: 'OK', 64: 'UT', 65: 'OK', 66: 'AR', 67: 'AR', 68: 'AR', 69: 'AR', 70: 'FL', 71: 'FL', 72: 'FL', 73: 'FL', 74: 'FL', 75: 'FL', 76: 'FL', 77: 'FL', 78: 'FL', 79: 'FL', 80: 'FL', 81: 'FL', 82: 'FL', 83: 'SC', 84: 'SC', 85: 'SC', 86: 'SC', 87: 'GA', 88: 'GA', 89: 'GA', 90: 'NC', 91: 'NC', 92: 'CO', 93: 'FL', 94: 'FL', 95: 'MN', 96: 'UT', 97: 'UT', 98: 'UT', 99: 'UT', 100: 'MD'

### 2. Data wrangling

In [31]:
# Define a function to determine the region
def determine_region(row):
    if row['Latitude'] > 40:
        region = 'North'
    else:
        region = 'South'
    
    if row['Longitude'] < -90:
        region = 'West'
    else:
        region = 'East'
    
    return f"{region}"

In [33]:
# Apply the function to create the Region column
frpp_data['Region'] = frpp_data.apply(determine_region, axis=1)

In [35]:
#  Calculate the total cost and cost efficiency to create a Cost Efficiency Column
frpp_data['Total_Cost'] = (
    frpp_data['Owned and Otherwise Managed Annual Operations Cost'] +
    frpp_data['Owned and Otherwise Managed Annual Maintenance Cost']
)


In [37]:
# Calculate the cost efficiency
frpp_data['Cost_Efficiency'] = frpp_data['Total_Cost'] / frpp_data['Square Feet (Buildings)']


In [39]:
# Display the first few rows with the new columns
print(frpp_data[['Latitude', 'Longitude', 'Region', 'Total_Cost', 'Cost_Efficiency']].head())

   Latitude  Longitude Region  Total_Cost  Cost_Efficiency
0  34.24330  -79.79580   East     6905.65         0.412278
1  32.82920  -83.73550   East     6905.65         0.306591
2  41.84585  -87.67311   East     6905.65         3.175011
3  28.02150  -81.73050   East     6905.65         0.941466
4  35.27560  -81.14500   East     6905.65         0.664005


In [41]:
# Select relevant columns for the analysis 'Region', 'Utilization', and 'Cost_Efficiency' are the columns of interest

columns_of_interest = ['Region', 'Utilization', 'Cost_Efficiency']
frpp_data_subset = frpp_data[columns_of_interest]


In [43]:
# Check for missing values
print(frpp_data_subset.isnull().sum())


Region             0
Utilization        0
Cost_Efficiency    0
dtype: int64


In [45]:
# Fill or drop missing values as necessary
frpp_data_subset = frpp_data_subset.dropna()

### 3. Aggregate Data by State

In [48]:
# Aggregating cost efficiency data by state
frpp_data['State'] = frpp_data.index.map(mapping_dict.get)

In [50]:
# Calculate cost efficiency as an example metric (e.g., Cost per Square Foot)
frpp_data['Cost per Square Foot'] = frpp_data['Owned and Otherwise Managed Annual Operations Cost'] / frpp_data['Square Feet (Buildings)']

In [52]:
# Aggregate data by state
state_cost_efficiency = frpp_data.groupby('State')['Cost per Square Foot'].mean().reset_index()


In [54]:
# Load state geometries (GeoJSON) to use with folium
geojson_path = 'C:/Users/lourd/A6_Open Project/Data Sets/us-states.json'

if os.path.exists(geojson_path):
    with open(geojson_path) as f:
        us_states_geojson = json.load(f)
else:
    print("GeoJSON file not found at the specified path.")


### 4. Create Choropleth Map

In [57]:
# Create a base map centered on the United States
m = folium.Map(location=[37.8, -96], zoom_start=4)

In [59]:
# Add choropleth layer
Choropleth(
    geo_data=us_states_geojson,
    name='choropleth',
    data=state_cost_efficiency,
    columns=['State', 'Cost per Square Foot'],
    key_on='feature.id',  # feature.id refers to the state abbreviation in GeoJSON
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Cost per Square Foot'
).add_to(m)


In [61]:
# Add layer control
LayerControl().add_to(m)

In [63]:
# Save map to an HTML file
m.save('C:/Users/lourd/A6_Open Project/Data Sets/Visuals/cost_efficiency_choropleth.html')


In [65]:
# Display map in notebook (optional, depending on environment)
m


### Results and Analysis: Higher cost per square foot in states like New York indicates lower cost efficiency, likely due to higher real estate prices and more intensive usage. Greater cost efficiency is seen in central and southern states such as Texas, Colorado, and Montana, where operational costs are lower relative to property size. The analysis suggests that factors like property size, maintenance costs, and regional real estate prices play a significant role in determining cost efficiency, raising new questions for further research such as How does property utilization affect cost efficiency? Exploring whether under-utilized or over-utilized properties influence the overall operational costs.